In [1]:
import time, onnx, torch, torchvision
import os, glob, torch, csv
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as utils

from torchvision import transforms
from PIL import Image

## DeepStart network definition

DeepStar consists of two layer of max pooled convolutional neural networks that is then fed into two layers of normal neural networks.

In [2]:
class DeepStar(nn.Module):
    def __init__(self):
        super(DeepStar, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=8, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.conv2 = nn.Conv2d(64, 32, kernel_size=2, stride=2, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #self.conv_dropout = nn.Dropout2d()

        self.fc1 = torch.nn.Linear(32 * 16 * 16 + 4, 64)
        self.fc2 = torch.nn.Linear(64 + 4, 2)
        
    def __name__(self):
        return "DeepStar"
    
    def get_loss(self):
        return torch.nn.L1Loss()
        
    def get_optim(self, rho, lr, weight_decay):
        return optim.Adadelta(self.parameters(), rho=rho, lr=lr, weight_decay=weight_decay)
        
    def forward(self, x):
        #pos_tensor = torch.tensor(x[0]).float().view(-1, 4)
        pos_tensor = x[0]
        
        print(1)
        x = F.relu(self.conv1(x[1]))
        x = self.pool1(x)
        print(2)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        #x = F.relu(self.conv3(x))
        #x = self.pool3(x)
        
        print(3)
        #x = self.conv_dropout(x)
        x = x.view(-1, 32 * 16 * 16)
        
        print(x)
        print(pos_tensor)
        x = torch.cat((x, pos_tensor), 1)
        
        print(4)
        x = F.relu(self.fc1(x))
        x = self.fc2(torch.cat((x, pos_tensor), 1))
        
        return x

## DataLoader

Because we use a custom dataset we have to have a custom data loader. The image has a heightmap in the first axis and the second and third axis represents the start and end point. The image name also contains the correct midpoint for that image.

In [3]:
class OldPathDataLoader(utils.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = glob.glob(data_dir + "*.png")
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file_path = self.files[idx]
        
        image_name = os.path.basename(file_path)
        img_as_img = Image.open(file_path)
        img_as_tensor = self.to_tensor(img_as_img)
        
        numbers = image_name.replace(".png","").split("_")
        
        x = float(numbers[0]) / 256.0
        y = float(numbers[1]) / 256.0
        
        return (img_as_tensor, [x, y])

In [4]:
class PathDataLoader(utils.Dataset):
    def __init__(self, data_dir):
        self.map = f'{data_dir}/map.png'
        self.data_path = f'{data_dir}/data.csv'
        self.to_tensor = transforms.ToTensor()
        self.data = pd.read_csv(self.data_path, encoding = "UTF-8")
    
    def __len__(self):
        return len(self.data["Start"])
    
    def __getitem__(self, idx):
        with Image.open(self.map) as img:
            sx, sy = self.to_tuple(self.data["Start"][idx])
            ex, ey = self.to_tuple(self.data["Stop"][idx])
            mx, my = self.to_tuple(self.data["Midpoint"][idx])

            img_tensor = self.to_tensor(img)
            
            sx, sy, ex, ey = 0.5

            return ([[sx / 256, sy / 256, ex / 256, ey / 256], img_tensor], (mx / 256, my / 256))
    
    def to_tuple(self, t):
        return tuple(map(int, t.replace('(','').replace(')', '').split(', '))) 